In [17]:
import os
import openai
from typing import Any, List
#from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from azure.ai.generative.index import get_langchain_retriever_from_index

from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.vectorstores import FAISS

Set credential values needed to authenciate and authorize access to your Azure OpenAI instance.

In [18]:
def setup_credentials():
    # Azure OpenAI credentials
    openai.api_type = os.environ["OPENAI_API_TYPE"]
    openai.api_key = os.environ["OPENAI_API_KEY"]
    openai.api_version = os.environ["OPENAI_API_VERSION"]
    openai.api_base = os.environ["OPENAI_API_BASE"]

    # Azure Cognitive Search credentials
    os.environ["AZURE_COGNITIVE_SEARCH_TARGET"] = os.environ["AZURE_AI_SEARCH_ENDPOINT"]
    os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = os.environ["AZURE_AI_SEARCH_KEY"]
    os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = os.environ["AZURE_AI_SEARCH_KEY"]
    os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = os.environ["AZURE_AI_SEARCH_INDEX_NAME"]

Execute the authorize with the given credentials above

In [19]:
setup_credentials()

In [20]:
from urllib.parse import urlparse

def get_search_resource_name(srv_url):
    val = urlparse(srv_url)
    return val.netloc
endpoint = get_search_resource_name(os.getenv('AZURE_AI_SEARCH_ENDPOINT'))
resource_name =str(endpoint).split('.', 1)[0]


In [21]:
from langchain.retrievers import AzureCognitiveSearchRetriever
retriever = AzureCognitiveSearchRetriever(content_key="content", top_k=10, service_name=resource_name)

Set you cognitive search endpoint and key for the vector store

Define the AzureSearch instance with the OpenAIEmbeddings to vectorize the input sample text data

Construct your prompt by specifying what the chat assistant does as well as the scope and domain of topics it can provide information for.  In addition, define any constraints, restricts or bounderies on how the prompt show behave.  The prompt template includes action the prompt can take; chat history of the system/user dialogue; context of the chat conversation.

In [22]:
template = """
System:
You are an AI assistant helping users with queries related to outdoor outdooor/camping gear and clothing.
Use the following pieces of context to answer the questions about outdoor/camping gear and clothing as completely, correctly, and concisely as possible.

---

{chat_history}

------

{context}

------


Question: {question}

Answer:"
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=[
        "context",
        "chat_history",
        "question"
    ],
)

In [23]:
memory = ConversationBufferMemory(memory_key="chat_history")

Initialize the AzureChatOpenAI instance with your Azure Open AI model and deployment.  The temperature value ranges from 0 to 1.  Value close to 0 denote how specific you want the response to be and value close to 1 denote how randomly you want the responses to be.

In [24]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT"],
    model_name=os.environ["AZURE_OPENAI_CHAT_MODEL"],
    temperature=0.7
)

Now, you can ready to invoke the key of langchain you want to use.  <provide the type of conversion you want>.  To activate the instance you need your LLM model to retrieve response, the promt template including the rules, and verbose <???>

In [25]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [26]:
qna_chain = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=retriever,
                                           condense_question_prompt=prompt_template,
                                           #return_source_documents=True,
                                           verbose=False,
                                           memory=memory)

Let's test the chat with a question!

In [27]:
#when you want to initalize chat with history.

inquiry = "What of your sleeping bags are polyester?"
response = qna_chain({"question": inquiry})
response['answer']

'The CozyNights Sleeping Bag (item_number: 7) and the MountainDream Sleeping Bag (item_number: 14) are both made of polyester.'